# Basic Python Training
### Python and pandas is extremely well documented. When in doubt, use Google and find the link corresponding to the API here https://pandas.pydata.org/docs/reference/index.html#api

In [ ]:
# If the below code doesn't work uncomment this and run (this adds the parent directory to the path)
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

In [ ]:
import matplotlib.pyplot as plt
from BasicSetupUtilities.MetaDataBuilder import CountryMetaDataFile
from DataIOUtilities.DataLib import DataLib, DatastreamPulls
import pandas as pd
import numpy as np

## Reading and writing/storing data

### Set up the data library 

In [ ]:
dl = DataLib("SignalData")

### Pull and look at the data. See SignalDataLibrary.py for the data available to you from Eikon. Use head and tail to look at the top and bottom of the dataframes.

In [ ]:
LongRates = dl.pull('LongRates')
ShortRates = dl.pull('ShortRates')

In [ ]:
LongRates.head()

In [ ]:
ShortRates.head()

### Generate some new data and store it

In [ ]:
LongMinusShortRates = LongRates - ShortRates
dl.write_data("LongMinusShortRates", LongMinusShortRates)
LongMinusShortRates.tail()

In [ ]:
LongMinusShortRatesFromStorage = dl.pull("LongMinusShortRates")
LongMinusShortRatesFromStorage.tail()

## Basic Data Access

### Select a subset of columns

In [ ]:
LongRates[['AUS', 'DEU']].tail()

### Select a subset of data by index name using loc (see the documentation if you're unfamiliar with the ':' operation for slicing)

In [ ]:
# Select a list of specific rows
LongRates.loc[[pd.Period('2020-03'), pd.Period('2020-04')], ['AUS', 'DEU']]

In [ ]:
# Select a slice of rows
LongRates.loc[pd.Period('2020-03'):pd.Period('2020-07'),['AUS', 'DEU']]

### Select a subset of data by index using iloc 

In [ ]:
# Select the last 5 elements
LongRates.iloc[-5:, :]

## Plotting the data. For this, look up matplotlib documentation

### Plotting all the columns of the dataframe

In [ ]:
BondReturnIndex = dl.pull('BondRetIdx/LocalFX')
BondReturnIndex.plot()
plt.xlabel('Date')
plt.ylabel('Return Index')
plt.title('Bond Return Index By Country')
plt.show()

### Plot series from different dataframes

In [ ]:
country = 'USA'
LongRates[country].plot()
ShortRates[country].plot()
LongMinusShortRates[country].plot()
plt.legend(['Long Rate', 'Short Rate', 'Long - Short'])
plt.xlabel('Date')
plt.title(f'{country} Long vs. Short Rate')
plt.show()

### Plot multiple charts at the same time. If you want them to all be part of the same figure, check out subplots https://matplotlib.org/api/_as_gen/matplotlib.pyplot.subplot.html

In [ ]:
countries = ['USA', 'AUS']
for country in countries:
    LongRates[country].plot()
    ShortRates[country].plot()
    LongMinusShortRates[country].plot()
    plt.legend(['Long Rate', 'Short Rate', 'Long - Short'])
    plt.xlabel('Date')
    plt.title(f'{country} Long vs. Short Rate')
    plt.show()

### Plot on separate axes (look up subplots)

In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
ax1_label = 'Long Minus Short Rate'
ax1.plot(LongMinusShortRates['USA'].to_timestamp(), label = ax1_label)
ax1.set_ylabel(ax1_label)
ax1.legend()
ax2_label = 'Bond Return Index'
ax2.plot(BondReturnIndex['USA'].to_timestamp(), color = 'tab:red', label = ax2_label)
ax2.set_ylabel(ax2_label)
ax2.legend()
plt.title('Bond Return Index vs. Long Minus Short Rate')
plt.show()

# Useful Operations. (See the time series section of the API)

### Changes (Arithmetic)

In [ ]:
LongRatesMonthlyChange = LongRates.diff()
LongRatesMonthlyChange['USA'].plot()
plt.xlabel("Date")
plt.title("Monthly Change in Long Rate (Arithmetic)")
plt.show()

In [ ]:
WEEKDAYS_IN_YEAR = 261


RGDP = dl.pull('GDP/Real')
RGDPChangeQoQ_Unannualized = RGDP.pct_change(1, fill_method = None).dropna(how='all')
RGDPChangeQoQ = (RGDPChangeQoQ_Unannualized + 1) ** 4 - 1

BondReturnsPastQuarter = BondReturnIndex.pct_change(int(WEEKDAYS_IN_YEAR / 4))

fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
ax1_label = 'RGDP Change, QoQ'
ax1.plot(RGDPChangeQoQ['USA'].to_timestamp(), color = 'tab:blue', label = ax1_label)
ax1.set_ylabel(ax1_label)
ax1.legend(loc=2)
ax2_label = 'Bond Return, Past Quarter'
ax2.plot(BondReturnsPastQuarter['USA'].to_timestamp(), color = 'tab:red', label = ax2_label)
ax2.set_ylabel(ax2_label)
ax2.legend()
plt.title('RGDP Change vs. Bond Returns')
plt.show()

### Rolling Measures (mean and standard deviation)

In [ ]:
RGDPChangeRollingMean = RGDPChangeQoQ.rolling(40, min_periods=20).mean() # 
RGDPChangeRollingMean.plot()
plt.title('GDP Growth Rolling Mean (20 Year)')
plt.show()
RGDPChangeRollingStdDev = RGDPChangeQoQ.rolling(40, min_periods=20).std()
RGDPChangeRollingStdDev.plot()
plt.title('GDP Growth Rolling Std Dev (20 Year)')
plt.show()

### Working with different frequencies

### Upsampling

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format
RGDPChange_Upsampled = RGDPChangeQoQ.resample('1B').ffill()
RGDPChange_Upsampled.head()

### Downsampling

In [ ]:
RGDPChange_Downsampled = RGDPChangeQoQ.resample('1Y').mean()
RGDPChange_Downsampled.head()

### Aggregations across columns

In [ ]:
all_countries = RGDPChangeQoQ.columns
RGDPChangeMeans = RGDPChangeQoQ.copy()
RGDPChangeMeans['Equal Weighted Mean'] = RGDPChangeQoQ.mean(axis=1)
RGDPChangeMeans.tail()

In [ ]:
random_weights = {country: np.random.uniform() for country in all_countries}
RGDPChangeMeans['Random Weighted Mean'] = sum([random_weights[country] * RGDPChangeQoQ[country] for country in countries])
RGDPChangeMeans['Random Weighted Mean'] /= sum(random_weights.values()) # Need to either do this or normalize the weights
RGDPChangeMeans.tail()

## Exercise
### Calculate the rolling 1-year return of a world bond portfolio that is weighted by GDP of each country. Say that you instantly rebalance at the beginning of each quarter with the GDP from last quarter.